## Implementing a ResNet-34 CNN Using Keras

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D

from functools import partial


DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)


class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)

        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []

        # para que la entrada tenga las mismas dimensiones de la salida y poder concatenarlas
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        # conectar las capas principales  entre si
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs

        # conectar las capas de skip entre si
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        # sumar la entrada con la salida de la capa principal y pasarlas por ReLU
        return self.activation(Z + skip_Z)


Now, we can build a **RestNet-34** using a Sequential model 

In [8]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))

prev_filters = 64

# Residual Nets
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

# Dense Nets
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

# el modlos que acabamos de implementar se encuentra en :
keras.applications.resnet.ResNet50()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_144 (Conv2D)         (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization_144 (Ba  (None, 112, 112, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_4 (Activation)   (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 residual_unit_64 (ResidualU  (None, 56, 56, 64)       74240     
 nit)                                                            
                                                      

Cada modelo pre-trained tiene el método `preprocess_input()` para procesar las imagenes y devolverlas en el formato de entrada del modelo

In [1]:
import numpy as np
from sklearn.datasets import load_sample_image
import matplotlib.pyplot as plt

# Load sample images
china = load_sample_image("china.jpg") / 255
china.shapes


(427, 640, 3)

## Trabajar con GPU

In [3]:
import sys
import tensorflow as tf
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")
print(tf.__version__)

No GPU was detected. CNNs can be very slow without a GPU.
2.10.0


In [ ]:


# Configura TensorFlow para utilizar la GPU 0
physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0], 'GPU')

fine_tune_epochs = 15  # 10
total_epochs = history_mobile_net2.epoch[-1] + fine_tune_epochs


# Crea una sesión de TensorFlow que se ejecute en la GPU 0
with tf.device('/GPU:0'):
    history_mobile_net2 = model.fit(train_ds,
                                    epochs=total_epochs,
                                    initial_epoch=history_mobile_net2.epoch[-1],
                                    validation_data=val_ds)


fig_fine_tune = graph_accuracy(history_mobile_net2, total_epochs,
                               model_name="Mobil_netV2_plus", create_new_graph=False, fig=fig)
